## Import all packages and functions needed

In [3]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

## Decision tree class

In [17]:
class dtree:
    """ A basic Decision Tree"""

    def __init__(self):
        """ Constructor """

    def read_data(self, filename):
        fid = open(filename, "r")
        data = []
        d = []
        for line in fid.readlines():
            d.append(line.strip())
        for d1 in d:
            data.append(d1.split(","))
        fid.close()

        self.featureNames = data[0]
        self.featureNames = self.featureNames[:-1]
        data = data[1:]
        self.classes = []
        for d in range(len(data)):
            self.classes.append(data[d][-1])
            data[d] = data[d][:-1]

        return data, self.classes, self.featureNames

    def classify(self, tree, datapoint):

        if type(tree) == type("string"):
            # Have reached a leaf
            return tree
        else:
            a = list(tree.keys())[0]
            for i in range(len(self.featureNames)):
                if self.featureNames[i] == a:
                    break
            try:
                t = tree[a][datapoint[i]]
                return self.classify(t, datapoint)
            except:
                return None

    def classifyAll(self, tree, data):
        results = []
        for i in range(len(data)):
            results.append(self.classify(tree, data[i]))
        return results

    def make_tree(self,
                  data,
                  classes,
                  featureNames,
                  maxlevel=-1,
                  level=0,
                  forest=0):
        """ The main function, which recursively constructs the tree"""

        nData = len(data)
        nFeatures = len(data[0])

        try:
            self.featureNames
        except:
            self.featureNames = featureNames

        # List the possible classes
        newClasses = []
        for aclass in classes:
            if newClasses.count(aclass) == 0:
                newClasses.append(aclass)

        # Compute the default class (and total entropy)
        frequency = np.zeros(len(newClasses))

        totalEntropy = 0
        totalGini = 0
        index = 0
        for aclass in newClasses:
            frequency[index] = classes.count(aclass)
            totalEntropy += self.calc_entropy(float(frequency[index]) / nData)
            totalGini += (float(frequency[index]) / nData)**2

            index += 1

        totalGini = 1 - totalGini
        default = classes[np.argmax(frequency)]

        if nData == 0 or nFeatures == 0 or (maxlevel >= 0
                                            and level > maxlevel):
            # Have reached an empty branch
            return default
        elif classes.count(classes[0]) == nData:
            # Only 1 class remains
            return classes[0]
        else:

            # Choose which feature is best
            gain = np.zeros(nFeatures)
            ggain = np.zeros(nFeatures)
            featureSet = list(range(nFeatures))
            if forest != 0:
                np.random.shuffle(featureSet)
                featureSet = featureSet[0:forest]
            for feature in featureSet:
                g, gg = self.calc_info_gain(data, classes, feature)
                gain[feature] = totalEntropy - g
                ggain[feature] = totalGini - gg

            #changed this
            bestFeature = np.argmax(ggain)
            
            tree = {featureNames[bestFeature]: {}}

            # List the values that bestFeature can take
            values = []
            for datapoint in data:
                if datapoint[feature] not in values:
                    values.append(datapoint[bestFeature])

            for value in values:
                # Find the datapoints with each feature value
                newData = []
                newClasses = []
                index = 0
                for datapoint in data:
                    if datapoint[bestFeature] == value:
                        if bestFeature == 0:
                            newdatapoint = datapoint[1:]
                            newNames = featureNames[1:]
                        elif bestFeature == nFeatures:
                            newdatapoint = datapoint[:-1]
                            newNames = featureNames[:-1]
                        else:
                            newdatapoint = datapoint[:bestFeature]
                            # newdatapoint.append(datapoint[bestFeature+1:])
                            newdatapoint = np.append(
                                newdatapoint, datapoint[bestFeature + 1:])
                            newNames = featureNames[:bestFeature]
                            # newNames.append(featureNames[bestFeature+1:])
                            newNames = np.append(
                                newNames, featureNames[bestFeature + 1:])
                        newData.append(newdatapoint)
                        newClasses.append(classes[index])
                    index += 1

                # Now recurse to the next level
                subtree = self.make_tree(newData, newClasses, newNames,
                                         maxlevel, level + 1, forest)

                # And on returning, add the subtree on to the tree
                tree[featureNames[bestFeature]][value] = subtree

            return tree

    def printTree(self, tree, name):
        if type(tree) == dict:
            print(name, tree.keys()[0])
            for item in tree.values()[0].keys():
                print(name, item)
                self.printTree(tree.values()[0][item], name + "\t")
        else:
            print(name, "\t->\t", tree)

    def calc_entropy(self, p):
        if p != 0:
            return -p * np.log2(p)
        else:
            return 0

    def calc_info_gain(self, data, classes, feature):

        # Calculates the information gain based on both entropy and the Gini impurity
        gain = 0
        ggain = 0
        nData = len(data)

        # List the values that feature can take
        values = []
        for datapoint in data:
            if datapoint[feature] not in values:
                values.append(datapoint[feature])

        featureCounts = np.zeros(len(values))
        entropy = np.zeros(len(values))
        gini = np.zeros(len(values))
        valueIndex = 0
        
        # Find where those values appear in data[feature] and the corresponding class
        for value in values:
            dataIndex = 0
            newClasses = []
            for datapoint in data:
                if datapoint[feature] == value:
                    featureCounts[valueIndex] += 1
                    newClasses.append(classes[dataIndex])
                dataIndex += 1

            # Get the values in newClasses
            classValues = []
            for aclass in newClasses:
                if classValues.count(aclass) == 0:
                    classValues.append(aclass)

            classCounts = np.zeros(len(classValues))
            classIndex = 0
            for classValue in classValues:
                for aclass in newClasses:
                    if aclass == classValue:
                        classCounts[classIndex] += 1
                classIndex += 1

            for classIndex in range(len(classValues)):
                entropy[valueIndex] += self.calc_entropy(
                    float(classCounts[classIndex]) / np.sum(classCounts))
                gini[valueIndex] += (
                    float(classCounts[classIndex]) / np.sum(classCounts))**2

            # Computes both the Gini gain and the entropy
            gain = gain + float(
                featureCounts[valueIndex]) / nData * entropy[valueIndex]
            ggain = ggain + float(
                featureCounts[valueIndex]) / nData * gini[valueIndex]
            valueIndex += 1
        print(gain)
        return gain, 1 - ggain
    

## Testing decision tree

In [18]:
data = [[0, 0], [1, 0], [0, 1], [1, 1]]
classes = [0, 1, 1, 1]
names = ['x1', 'x2']
tree = dtree().make_tree(data, classes, names)
tree

0.5
0.5
0.0
0.0


{'x1': {0: {'x2': {0: 0, 1: 1}}, 1: 1}}

# Assessment

Because the implementation was given to us, I didn't make many changes to the code. I changed the best feature value to consider the Gini gain, rather than the entropy. This sped the overall function up by allowing it to be more efficent.

## Random forest class

In [7]:
class randomforest:
    """The random forest algorithm based on the decision tree of Chapter 6"""

    def __init__(self):
        """ Constructor """
        self.tree = dtree()

    def rf(self,
           data,
           targets,
           features,
           nTrees,     #Change this
           nSamples,
           nFeatures,  #Change this
           maxlevel=5):

        nPoints = np.shape(data)[0]
        nDim = np.shape(data)[1]
        self.nSamples = nSamples
        self.nTrees = nTrees

        classifiers = []

        for i in range(nTrees):
            print(i)
            # Compute bootstrap samples
            samplePoints = np.random.randint(0, nPoints, (nPoints, nSamples))

            for j in range(nSamples):
                sample = []
                sampleTarget = []
                for k in range(nPoints):
                    sample.append(data[samplePoints[k, j], :])
                    sampleTarget.append(targets[samplePoints[k, j]])
            # Train classifiers
            classifiers.append(
                self.tree.make_tree(
                    sample, sampleTarget, features, maxlevel,
                    forest=nFeatures))
        return classifiers

    def rfclass(self, classifiers, data):

        decision = []
        # Majority voting
        for j in range(len(data)):
            outputs = []
            #print data[j]
            for i in range(self.nTrees):
                out = self.tree.classify(classifiers[i], data[j, :])
                if out is not None:
                    outputs.append(out)
            # List the possible outputs
            out = []
            for each in outputs:
                if out.count(each) == 0:
                    out.append(each)
            frequency = np.zeros(len(out))

            index = 0
            if len(out) > 0:
                for each in out:
                    frequency[index] = outputs.count(each)
                    index += 1
                decision.append(out[frequency.argmax()])
            else:
                decision.append(None)
        return decision

## Testing random forest

In [29]:
# import data
breast = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data", header=None)
print(breast.shape)
breast.head(10)

(569, 32)


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678
5,843786,M,12.45,15.70,82.57,477.1,0.12780,0.17000,0.15780,0.08089,...,15.47,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440
6,844359,M,18.25,19.98,119.60,1040.0,0.09463,0.10900,0.11270,0.07400,...,22.88,27.66,153.20,1606.0,0.1442,0.2576,0.3784,0.1932,0.3063,0.08368
7,84458202,M,13.71,20.83,90.20,577.9,0.11890,0.16450,0.09366,0.05985,...,17.06,28.14,110.60,897.0,0.1654,0.3682,0.2678,0.1556,0.3196,0.11510
8,844981,M,13.00,21.82,87.50,519.8,0.12730,0.19320,0.18590,0.09353,...,15.49,30.73,106.20,739.3,0.1703,0.5401,0.5390,0.2060,0.4378,0.10720
9,84501001,M,12.46,24.04,83.97,475.9,0.11860,0.23960,0.22730,0.08543,...,15.09,40.68,97.65,711.4,0.1853,1.0580,1.1050,0.2210,0.4366,0.20750


In [34]:
X = breast.drop([0,1], axis=1).values
y = breast[1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [36]:
rf = randomforest()
#Change 4 and 6
rf_model = rf.rf(X_train, y_train, list(range(1, 31)), 40, 500, 7, maxlevel=3)
out = rf.rfclass(rf_model, X_test)

0
0.06673522632219842
0.027174786761758857
0.043190075723963134
0.019241510993765865
0.004395604395604396
0.006054697806952679
0.006054697806952679
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1
0.02090060440511415
0.026955302212066826
0.0
0.013987973574945672
0.023637115389370262
0.0
0.013186813186813187
2
0.04309811093282816
0.0159233241710693
0.022779182366154463
0.0
0.02969181319632294
0.026373626373626374
0.020318928566673694
3
0.11462632509543107
0.006054697806952679
0.007132115379860508
0.02255969781646243
0.008791208791208791
0.04951500002852925
0.004395604395604396
0.0
0.0
0.0
0.0
0.0
0.0
0.0
4
0.14423220418778093
0.02112008895480618
0.008791208791208791
0.08351648351648351
0.0
0.059603110942337896
0.037936538305142944


# Compared to Project 2

In this one, I was tasked with changing the number of trees (nTrees), and number of features (nFeatures) being used in the implementation. I have changed them to what I believe is also an efficient accuracy score, considering the model. Compared to project 2 (which ran on the CIFAR-10 data set) it did not do as well. I was able to get a 93% accuracy rating one of my project 2 implementations. I did not achieve that here. 
